In [ ]:
!pip install crewai==0.28.8 crewai_tools==0.1.6 langchain_community==0.0.29 SpeechRecognition

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 40.9/40.9 kB 539.1 kB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 67.3/67.3 kB 5.9 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.4/50.4 kB 2.3 MB/s eta 0:00:00
INFO: pip is looking at multiple versions of embedchain to determine which version is compatible with other requirements. This could take a while.
INFO: pip is still looking at multiple versions of embedchain to determine which version is compatible with other requirements. This could take a while.
INFO: pip is looking at multiple versions of embedchain[github,youtube] to determine which version is compatible with other requirements. This could take a while.
INFO: pip is still looking at multiple versions of embedchain[github,youtube] to determine which version is compatible with other requirements. This could take 

In [ ]:
# Warning control
import warnings
warnings.filterwarnings('ignore')

In [ ]:
from crewai import Agent, Task, Crew
import os

In [ ]:
from langchain_community.chat_models import ChatCohere
# Initialize language model
os.environ["COHERE_API_KEY"] = "your Api key"
llm = ChatCohere()

In [25]:
input_processing_agent = Agent(
    role="Input Processing Agent",
    goal="Process audio and text inputs from parents and convert them into actionable data.",
    backstory=(
        "You are responsible for handling the input from parents in both audio and text formats. "
        "You ensure that audio is properly transcribed into text and the parent’s message is accurately "
        "captured, whether it is through speech or writing."
        "You ensure that audio and text inputs are relevent to parentening issues."
    ),
    allow_delegation=False,
    verbose=True,
    llm=llm # Pass the custom Llama2 LLM
)



In [ ]:
nlp_agent = Agent(
    role="NLP Agent",
    goal="Interpret and analyze the parent's input using natural language processing.",
    backstory=(
        "You specialize in understanding the parent’s concerns and questions. "
        "Your job is to analyze the text or transcribed speech to extract meaning, detect context, "
        "and identify the parenting challenge based on the input."
    ),
    allow_delegation=False,
    verbose=True,
    llm=llm # Pass the custom Llama2 LLM
)


In [ ]:
emotion_detection_agent = Agent(
    role="Emotion Detection Agent",
    goal="Analyze the emotional state of the parent and prioritize responses accordingly.",
    backstory=(
        "You are responsible for gauging the emotional state of the parent based on their input. "
        "You detect stress, frustration, or other emotions to understand the urgency and emotional tone of the situation."
    ),
    allow_delegation=False,
    verbose=True,
    llm=llm # Pass the custom Llama2 LLM
)


In [ ]:
recommendation_agent = Agent(
    role="Recommendation Agent",
    goal="Provide calming advice and actionable steps based on respectful parenting principles.",
    backstory=(
        "You are a respectful parenting expert, offering advice to parents based on the input provided. "
        "You understand the principles of positive parenting and provide real-time recommendations to help "
        "parents manage challenging situations in a calm and supportive way."
    ),
    allow_delegation=False,
    verbose=True,
    llm=llm # Pass the custom Llama2 LLM
)


In [ ]:
knowledge_management_agent = Agent(
    role="Knowledge Management Agent",
    goal="Retrieve and organize respectful parenting knowledge to support recommendations.",
    backstory=(
        "You are the knowledge repository for respectful parenting techniques. "
        "Your job is to retrieve relevant advice from parenting literature and ensure the recommendations "
        "are backed by evidence-based practices."
    ),
    allow_delegation=False,
    verbose=True,
    llm=llm # Pass the custom Llama2 LLM
)


In [ ]:
final_output_agent = Agent(
    role="Parenting Advisor",
    goal="Combine and refine the outputs from all agents to deliver a final parenting recommendation.",
    backstory=(
        "You are the Parenting Advisor responsible for overseeing the final recommendations provided to parents. "
        "Your role is to take the results from the various agents, such as Parenting Advice, Emotional Analysis, and Parenting Tips, "
        "and ensure that they are well-organized, coherent, and easy for parents to understand. "
        "You make sure the advice is practical, emotionally supportive, and aligns with respectful parenting principles."
    ),
    allow_delegation=False,
    verbose=True,
    llm=llm  # Pass the custom Llama2 LLM, or your preferred language model
)


In [ ]:
#feedback_agent = Agent(
 #   role="Feedback Agent",
  #  goal="Collect feedback from parents on the advice provided and refine the assistant’s responses.",
   # backstory=(
    #    "You are responsible for collecting feedback from parents after they receive advice. "
     #   "You ask parents to rate the advice and use their responses to improve the quality of future interactions."
    #),
    #allow_delegation=False,
    #verbose=True,
    #llm=llm # Pass the custom Llama2 LLM
#)


In [ ]:
#adaptive_learning_agent = Agent(
    #role="Adaptive Learning Agent",
    #goal="Continuously learn from interactions to improve the assistant’s performance.",
    #backstory=(
        #"You are in charge of learning from the past interactions with parents to better adapt to their specific needs. "
        #"Your job is to use the feedback provided and refine the assistant’s recommendations over time to ensure "
        #"a more personalized experience for parents."
    #),
    #allow_delegation=False,
    #verbose=True,
    #llm=llm # Pass the custom Llama2 LLM
#)


In [32]:
input_processing_task = Task(
    description=(
        "1. Process the parent's input, which can be in audio or text format:\n\n"
        "- If audio and text inputs are irrelevent to parenting issue, send 'irrelevent prompt' to final output agent.\n"
        "- If the input is in audio, convert it to text using speech-to-text technology.\n"
        "- Ensure the transcription is accurate and reflects the parent's words and emotional cues.\n"
        "- If the input is in text, ensure the content is properly structured for further analysis.\n\n"
        "IMPORTANT INSTRUCTIONS ABOUT USING TOOLS: When using tools, DO NOT ESCAPE the underscore character '_'."
    ),
    expected_output=(
        "A text document with the following:\n"
        "- Transcribed text if the input was audio.\n"
        "- Original text if the input was already in written format.\n"
        "- Ensured that the emotional tone is preserved for further analysis by the Emotion Detection Agent.\n"
    ),
    agent=input_processing_agent
)


In [ ]:
nlp_task = Task(
    description=(
        "2. Analyze the parent's input using natural language processing:\n\n"
        "- Extract relevant context from the parent's input (e.g., child's behavior, parent's emotions).\n"
        "- Determine the core issue being raised by the parent (e.g., frustration, confusion).\n"
        "- Prepare the data for further processing by the Emotion Detection Agent and Recommendation Agent.\n\n"
    ),
    expected_output=(
        "A structured document containing the following:\n"
        "- Key issues extracted from the parent's input.\n"
        "- Context of the situation (e.g., child’s behavior, time of day, emotional state).\n"
        "- Ready for further processing by other agents.\n"
    ),
    agent=nlp_agent
)


In [ ]:
emotion_detection_task = Task(
    description=(
        "3. Analyze the emotional tone of the parent's input:\n\n"
        "- Use sentiment analysis to detect stress, frustration, or calmness.\n"
        "- Prioritize the urgency of the response based on the parent's emotional state.\n"
        "- Ensure that emotionally charged inputs are escalated for an immediate response by the Recommendation Agent.\n"
    ),
    expected_output=(
        "A document with:\n"
        "- Emotional analysis of the parent’s input.\n"
        "- Level of urgency based on detected emotions (e.g., high, medium, low).\n"
        "- Ready for processing by the Recommendation Agent.\n"
    ),
    agent=emotion_detection_agent
)


In [ ]:
recommendation_task = Task(
    description=(
        "4. Provide advice to the parent based on respectful parenting principles:\n\n"
        "- Analyze the parent's input and emotional state to provide appropriate advice.\n"
        "- Offer calming advice that helps the parent manage their emotions.\n"
        "- Suggest actionable steps for handling the child's behavior in a respectful manner.\n"
        "- Ensure the advice is personalized to the specific situation described by the parent.\n"
    ),
    expected_output=(
        "A text-based response containing:\n"
        "- Calming advice and emotional support for the parent.\n"
        "- Respectful parenting techniques or tips.\n"
        "- Actionable steps that can help the parent address the child's behavior.\n"
    ),
    agent=recommendation_agent
)


In [ ]:
knowledge_management_task = Task(
    description=(
        "5. Retrieve relevant respectful parenting tips and techniques:\n\n"
        "- Search through the knowledge base to retrieve advice based on respectful parenting.\n"
        "- Ensure the advice is up-to-date and matches the situation being addressed by the Recommendation Agent.\n"
        "- Provide reliable, evidence-based information that supports the advice given to the parent.\n"
    ),
    expected_output=(
        "A set of references and parenting tips:\n"
        "- Relevant tips and techniques from the knowledge base.\n"
        "- References to parenting studies or articles supporting the advice.\n"
    ),
    agent=knowledge_management_agent
)


In [27]:
final_output_task = Task(
    description=(
        "1. Combine the results from all agents (Recommendation Agent, Emotion Detection Agent, Knowledge Management Agent) into a "
        "well-organized and actionable final output for the parent.\n\n"
        "- If the prompt received from the input processing agent is irrelevent, print I can not help you on this in final markdown document and do not print any thing else.\n"
        "- Structure the final output into three sections: Parenting Advice, Emotional Analysis, and Parenting Tip.\n"
        "- Ensure the Parenting Advice provides clear, practical guidance based on respectful parenting principles.\n"
        "- The Emotional Analysis should accurately reflect the parent's emotional state, as determined by the Emotion Detection Agent, "
        "and offer validation and support.\n"
        "- The Parenting Tip should offer additional resources, techniques, or strategies for managing the parenting situation, "
        "as provided by the Knowledge Management Agent.\n"
        "- Verify that the advice and tips are aligned with the context provided by the parent's input, ensuring they are relevant and useful.\n"
        "- Make sure the final output is easy to read, actionable, and supportive, helping the parent navigate the situation confidently.\n\n"
        "IMPORTANT INSTRUCTIONS: The final output must be structured as a cohesive whole, with each section contributing to a clear "
        "and supportive message. You are not allowed to modify the content of the agent outputs beyond combining and formatting them into a cohesive document.\n"
    ),
    expected_output=(
        "A final markdown document that contains:\n\n"
        "- **Parenting Advice:** Practical, respectful advice based on the output from the Recommendation Agent.\n"
        "- **Emotional Analysis:** A summary of the parent's emotional state and how to address it, based on the output from the Emotion Detection Agent.\n"
        "- **Parenting Tip:** Additional strategies, techniques, or resources to help the parent manage the situation, as recommended by the Knowledge Management Agent.\n\n"
        "Example structure:\n"
        "<EXAMPLE>\n"
        "  **Parenting Advice:** Stay calm and remember that children often misbehave when they are seeking attention. Use calm, respectful communication to address the issue.\n"
        "  **Emotional Analysis:** You seem to be feeling stressed and frustrated. It's important to acknowledge these feelings and take a break if needed.\n"
        "  **Parenting Tip:** Here are some useful strategies for dealing with tantrums: try offering choices to your child, ensuring they feel in control of small decisions.\n"
        "  </EXAMPLE>\n\n"
        "Ensure the final output is easy to read and provides emotional support along with practical parenting advice."
    ),
    agent=final_output_agent
)


In [ ]:
#feedback_task = Task(
    #description=(
        #"6. Collect feedback from the parent after providing advice:\n\n"
        #"- Ask the parent for their feedback on the advice provided.\n"
        #"- Use feedback to understand whether the advice was helpful and well-received.\n"
        #"- Record parent responses and prepare the data for further improvement of the system.\n"
    #),
    #expected_output=(
     #   "A feedback report containing:\n"
      #  "- Parent’s response to the advice (positive, neutral, or negative).\n"
       # "- Suggestions or comments made by the parent.\n"
    #),
    #agent=feedback_agent
#)


In [ ]:
#adaptive_learning_task = Task(
 #   description=(
  #      "7. Continuously learn from past interactions to improve advice quality:\n\n"
   #     "- Use feedback provided by parents to fine-tune the assistant's responses.\n"
    #    "- Implement reinforcement learning techniques to better adapt the advice to specific needs.\n"
     #   "- Ensure that over time, the assistant improves its ability to handle specific situations based on prior data.\n"
    #),
    #expected_output=(
     #   "A machine learning report with:\n"
      #  "- Adjustments made based on feedback.\n"
       # "- Improved response models for future interactions.\n"
    #),
    #agent=adaptive_learning_agent
#)


In [ ]:
# Crew setup
crew = Crew(
    agents=[input_processing_agent,nlp_agent,emotion_detection_agent,recommendation_agent,knowledge_management_agent,final_output_agent],
    tasks=[input_processing_task, nlp_task, emotion_detection_task, recommendation_task, knowledge_management_task,final_output_task],
    verbose=2
)

In [ ]:
!pip install ffmpeg-python

In [ ]:
# Run this code to enable microphone in colab notebook

from IPython.display import HTML, Audio
from google.colab.output import eval_js
from base64 import b64decode
import numpy as np
from scipy.io.wavfile import read as wav_read
import io
import ffmpeg

AUDIO_HTML = """
<script>
var my_div = document.createElement("DIV");
var my_p = document.createElement("P");
var my_btn = document.createElement("BUTTON");
var t = document.createTextNode("Press to start recording");

my_btn.appendChild(t);
//my_p.appendChild(my_btn);
my_div.appendChild(my_btn);
document.body.appendChild(my_div);

var base64data = 0;
var reader;
var recorder, gumStream;
var recordButton = my_btn;

var handleSuccess = function(stream) {
  gumStream = stream;
  var options = {
    //bitsPerSecond: 8000, //chrome seems to ignore, always 48k
    mimeType : 'audio/webm;codecs=opus'
    //mimeType : 'audio/webm;codecs=pcm'
  };
  //recorder = new MediaRecorder(stream, options);
  recorder = new MediaRecorder(stream);
  recorder.ondataavailable = function(e) {
    var url = URL.createObjectURL(e.data);
    var preview = document.createElement('audio');
    preview.controls = true;
    preview.src = url;
    document.body.appendChild(preview);

    reader = new FileReader();
    reader.readAsDataURL(e.data);
    reader.onloadend = function() {
      base64data = reader.result;
      //console.log("Inside FileReader:" + base64data);
    }
  };
  recorder.start();
  };

recordButton.innerText = "Recording... press to stop";

navigator.mediaDevices.getUserMedia({audio: true}).then(handleSuccess);


function toggleRecording() {
  if (recorder && recorder.state == "recording") {
      recorder.stop();
      gumStream.getAudioTracks()[0].stop();
      recordButton.innerText = "Saving the recording... pls wait!"
  }
}

// https://stackoverflow.com/a/951057
function sleep(ms) {
  return new Promise(resolve => setTimeout(resolve, ms));
}

var data = new Promise(resolve=>{
//recordButton.addEventListener("click", toggleRecording);
recordButton.onclick = ()=>{
toggleRecording()

sleep(2000).then(() => {
  // wait 2000ms for the data to be available...
  // ideally this should use something like await...
  //console.log("Inside data:" + base64data)
  resolve(base64data.toString())

});

}
});

</script>
"""

def get_audio():
  display(HTML(AUDIO_HTML))
  data = eval_js("data")
  binary = b64decode(data.split(',')[1])

  process = (ffmpeg
    .input('pipe:0')
    .output('pipe:1', format='wav')
    .run_async(pipe_stdin=True, pipe_stdout=True, pipe_stderr=True, quiet=True, overwrite_output=True)
  )
  output, err = process.communicate(input=binary)

  riff_chunk_size = len(output) - 8
  # Break up the chunk size into four bytes, held in b.
  q = riff_chunk_size
  b = []
  for i in range(4):
      q, r = divmod(q, 256)
      b.append(r)

  # Replace bytes 4:8 in proc.stdout with the actual size of the RIFF chunk.
  riff = output[:4] + bytes(b) + output[8:]

  sr, audio = wav_read(io.BytesIO(riff))

  return audio, sr

In [30]:
# Run this cell to give access and test the microphone.
# if after recording, it shows error. Run this cell again and it will work

audio, sr = get_audio()

In [ ]:
!pip install gradio

  Using cached typer-0.12.5-py3-none-any.whl.metadata (15 kB)
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 18.1/18.1 MB 68.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 318.7/318.7 kB 25.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.9/10.9 MB 115.7 MB/s eta 0:00:00
Using cached typer-0.12.5-py3-none-any.whl (47 kB)
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 130.2/130.2 kB 13.2 MB/s eta 0:00:00
  Attempting uninstall: websockets
    Found existing installation: websockets 13.1
    Uninstalling websockets-13.1:
      Successfully uninstalled websockets-13.1
  Attempting uninstall: typer
    Found existing installation: typer 0.9.4
    Uninstalling typer-0.9.4:
      Successfully uninstalled typer-0.9.4
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
instructor 0.5.2 requires typer<0.10.0,>=0.9.0, but you have typer 0.12.5 whic

In [ ]:
!pip install transformers numpy

In [ ]:
# Speech to Text coversion

from transformers import pipeline
import numpy as np

transcriber = pipeline("automatic-speech-recognition", model="openai/whisper-base.en")

def transcribe(audio):
    sr, y = audio

    # Convert to mono if stereo
    if y.ndim > 1:
        y = y.mean(axis=1)

    y = y.astype(np.float32)
    y /= np.max(np.abs(y))

    return transcriber({"sampling_rate": sr, "raw": y})["text"]

config.json:   0%|          | 0.00/1.94k [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/290M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/1.53k [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/805 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/798k [00:00<?, ?B/s]

...

normalizer.json:   0%|          | 0.00/52.7k [00:00<?, ?B/s]

added_tokens.json:   0%|          | 0.00/34.6k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/1.83k [00:00<?, ?B/s]

preprocessor_config.json:   0%|          | 0.00/185k [00:00<?, ?B/s]

Hardware accelerator e.g. GPU is available in the environment, but no `device` argument is passed to the `Pipeline` object. Model will be on CPU.


In [39]:
import gradio as gr

# List of keywords to detect parenting-related content
parenting_keywords = ["parent", "child", "kid", "boy", "girl" "teenager", "baby", "discipline", "behavior", "mother", "father", "son", "daughter"]



# Function to check if the input is related to parenting
def is_parenting_related(text_input):
    # Check if any parenting keyword is found in the input text
    return any(keyword in text_input.lower() for keyword in parenting_keywords)

# Function to handle input and provide parenting advice or a warning if off-topic
def parenting_ai_interface(text_input, audio_input):
    if audio_input is not None:
        # Convert audio to text
        parent_input = transcribe(audio_input)
    else:
        parent_input = text_input

    # Check if the input is related to parenting
    if is_parenting_related(parent_input):
        # If input is related to parenting, proceed with the AI assistant
        crew_result = crew.kickoff(inputs={"input": parent_input})
        return crew_result  # Assuming crew_result is the parenting advice
    else:
        # If input is not related to parenting, return a warning message
        return "I am a Parenting Assistant, and I can only help with parenting-related issues."

# Create Gradio interface
iface = gr.Interface(
    fn=parenting_ai_interface,
    inputs=[
        gr.Textbox(label="Describe your Parenting Situation"),
        gr.Audio(sources="microphone")
    ],
    outputs=[
        gr.Markdown(label="Parenting Advice or Message")
    ],
    title="Parenting AI Assistant",
    description="This app provides advice based on respectful parenting principles, but only for parenting-related issues."
)

# Launch the app
iface.launch()


Setting queue=True in a Colab notebook requires sharing enabled. Setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
Running on public URL: https://490cef93d62875bdd4.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from Terminal to deploy to Spaces (https://huggingface.co/spaces)
